## Load TEST data

In [14]:
import pandas as pd
import numpy as np
from collections import defaultdict
from pandas import DataFrame

In [15]:
#load positive data id
pos_ids = defaultdict(list)
lines = open("positive_data.txt", 'r').readlines()
for l in lines:
    es = l.strip().split()
    pos_ids[es[0]].append(es[1])
print(pos_ids)
print(np.sum([len(l) for l in pos_ids.values()]))

defaultdict(<class 'list'>, {'A': ['A10', 'B1', 'B2', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'H1', 'H2', 'H3', 'I10', 'J1', 'J2', 'J3', 'K1', 'K3', 'L10', 'M3', 'N2', 'N5', 'P1', 'P2', 'P3', 'Q5', 'Q10'], 'B': ['C7', 'C12', 'H1', 'H2', 'H3', 'I10', 'N2'], 'C': ['H1', 'H2', 'H3'], 'D': ['C4', 'H3'], 'E': ['C4', 'C7', 'M10', 'Q1', 'Q2', 'Q3'], 'F': ['C4', 'D10', 'H1', 'H3', 'M3'], 'G': ['B1', 'C5', 'C7', 'C9', 'I1', 'I3', 'J1', 'M10'], 'H': ['B1', 'E10', 'H4', 'J1', 'L7']})
63


In [16]:
#load energy data
all_data = pd.DataFrame()
for i in ["A","B","C","D","E","F","G","H"]:
    #print(i)
    df = pd.read_csv("ddg/test/U"+i+"_ddg_terms.txt", sep="\t")
    #delete all zero col
    df = df.loc[:,~(df==0).all(axis=0)]
    df.loc[len(df.index)] = ["B1"] + [0.0]*18
    
    df['UAA'] = i
    df['label'] = [t in pos_ids[i] for t in df["MUT"]]
    all_data = pd.concat([all_data, df], axis=0, ignore_index=True)
all_data = all_data.rename(columns={"MUT": "id"})

In [17]:
#remove duplicates
#I12 M11
#L8 I10
#J7 N3
all_data = all_data[all_data['id']!="M11" ]
all_data = all_data[all_data['id']!="I10" ]
all_data = all_data[all_data['id']!="N3" ]

In [18]:
#load ESM-1v, ESM-1b
df1v = pd.read_csv("../evotuning/mm_diskAB_1v_labeled.csv")
df1b = pd.read_csv("../evotuning/mm_diskAB_1b_labeled.csv")

In [19]:
df1v

,index,id,mutant,esm1v_t33_650M_UR90S_1,esm1v_t33_650M_UR90S_2,esm1v_t33_650M_UR90S_3,esm1v_t33_650M_UR90S_4,esm1v_t33_650M_UR90S_5
0,0,A1,S186S,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,A2,L309A|C348S|Y384F,-5.284741,-4.899321,-4.121393,-3.475363,-4.216409
2,2,A3,Y306A|Y384F,-5.284741,-4.899321,-4.121393,-3.475363,-4.216409
3,3,A4,Y306M|L309G|C348A,-2.775700,-2.103650,-2.359508,-2.893652,-2.797134
4,4,A5,Y384F,-5.284741,-4.899321,-4.121393,-3.475363,-4.216409
...,...,...,...,...,...,...,...,...
184,184,M12,L305F|L309V|N346E|C348G|Y384F|W417A,-7.422001,-7.790501,-9.690056,-4.338758,-7.223655
185,185,N1,L305C|L309M|N346Q|C348G|Y384W,-8.192837,-9.085862,-6.321147,-6.333002,-8.308035
186,186,N2,L305F|L309M|N346G|C348G|Y384W|W417A,-7.422001,-7.790501,-9.690056,-4.338758,-7.223655
187,187,N3,L305C|L309V|N346E|C348G|Y384W,-8.192837,-9.085862,-6.321147,-6.333002,-8.308035


In [20]:
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_1']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_2']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_3']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_4']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_5']], how='left', on='id')
all_data = pd.merge(all_data, df1b.loc[:, ['id','esm_msa1b_t12_100M_UR50S']], how='left', on='id')

In [21]:
all_data

,id,fa_rep:,ref:,hbond_sc:,fa_sol:,yhh_planarity:,dslf_fa13:,fa_intra_sol_xover4:,omega:,p_aa_pp:,...,fa_dun:,rama_prepro:,UAA,label,esm1v_t33_650M_UR90S_1,esm1v_t33_650M_UR90S_2,esm1v_t33_650M_UR90S_3,esm1v_t33_650M_UR90S_4,esm1v_t33_650M_UR90S_5,esm_msa1b_t12_100M_UR50S
0,K12,1.146,-0.043,5.221,0.000,0.000,0.016,0.000,0.000,2.203,...,0.000,0.000,A,False,-7.422001,-7.790501,-9.690056,-4.338758,-7.223655,-13.179728
1,K11,-0.495,3.148,2.325,0.000,0.000,-0.003,0.000,0.000,4.621,...,0.000,0.000,A,False,-7.422001,-7.790501,-9.690056,-4.338758,-7.223655,-13.179728
2,K10,-0.559,0.657,1.911,0.000,0.000,-0.023,0.000,0.000,1.611,...,0.000,0.000,A,False,-5.179635,-5.837735,-5.669314,-3.671014,-7.604808,-9.562101
3,M3,0.126,-0.283,1.720,0.001,-0.002,-0.021,-0.009,-0.003,1.165,...,0.021,-0.001,A,True,-8.192837,-9.085862,-6.321147,-6.333002,-8.308035,-11.596353
4,M2,0.011,-0.537,3.081,0.000,0.000,0.087,0.000,0.000,3.116,...,0.000,0.000,A,False,-5.284741,-4.899321,-4.121393,-3.475363,-4.216409,-7.240523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,J3,1.142,1.141,-0.732,0.000,0.000,-0.034,0.000,0.000,-2.428,...,0.000,0.000,H,False,-5.284741,-4.899321,-4.121393,-3.475363,-4.216409,-7.240523
484,M10,1.260,-1.203,1.863,-0.112,0.012,0.065,0.002,-0.003,1.063,...,0.125,-0.010,H,False,-5.179635,-5.837735,-5.669314,-3.671014,-7.604808,-9.562101
485,M12,0.161,5.190,-0.319,0.000,0.000,-0.023,0.000,0.000,0.569,...,0.000,0.000,H,False,-7.422001,-7.790501,-9.690056,-4.338758,-7.223655,-13.179728
486,M1,1.873,-1.680,-0.139,0.000,0.000,-0.028,0.000,0.000,-3.994,...,0.000,0.000,H,False,-5.284741,-4.899321,-4.121393,-3.475363,-4.216409,-7.240523


### unirep

In [22]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [23]:
i2s = {}
lines = open("map_label_fname_test.txt", 'r')
for l in lines:
    es = l.strip().split()
    i2s[es[0]] = es[2]

In [24]:
sequences = [i2s[t] for t in i2s.keys()]
s2t = {i2s[t]:t for t in i2s.keys()}

In [49]:
#load eUnirep 1900/256/64 embedding
from jax_unirep.evotuning_models import mlstm1900
from jax_unirep.utils import load_params
from jax_unirep import get_reps

evo_tag = "1900"
evo_num = int(evo_tag)
params = load_params(folderpath='../evotuning/unirep/weights/'+evo_tag+'/', paper_weights=evo_num)[1]
h_avg, h_final, c_final= get_reps(sequences, params=params,mlstm_size=evo_num)

In [50]:
df_evo = pd.DataFrame()
for i, t in enumerate(i2s.keys()):
    d = pd.Series([s2t[sequences[i]]]+[x for x in h_avg[i]])
    assert(t == s2t[sequences[i]])
    df_evo = pd.concat([df_evo, d.to_frame().T], axis=0, ignore_index=True)

In [51]:
convert = {i:"e"+str(i) for i in range(1+evo_num)}
convert[0] = "id"
df_evo = df_evo.rename(columns=convert)

In [52]:
df_evo

,id,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e1891,e1892,e1893,e1894,e1895,e1896,e1897,e1898,e1899,e1900
0,I1,0.058345,-0.01952,0.35041,-0.028903,-0.086466,0.143834,-0.043299,-0.068313,-0.183652,...,0.140327,-0.023228,0.206749,-0.43469,-0.058584,0.07372,0.114995,0.178099,0.018919,0.485972
1,I2,0.055378,-0.01976,0.344391,-0.023888,-0.079507,0.139645,-0.044189,-0.066993,-0.176953,...,0.124441,-0.021314,0.204665,-0.453012,-0.044183,0.078256,0.110084,0.180214,0.017755,0.510337
2,I3,0.060902,-0.022181,0.348113,-0.029246,-0.084876,0.14864,-0.041625,-0.068084,-0.177198,...,0.139996,-0.021838,0.202956,-0.434582,-0.067029,0.072917,0.117284,0.180058,0.007401,0.503037
3,I4,0.05775,-0.020967,0.341295,-0.02648,-0.079342,0.146172,-0.04183,-0.067086,-0.17238,...,0.128645,-0.02063,0.198689,-0.450626,-0.065011,0.088416,0.113538,0.182175,0.007792,0.499669
4,I5,0.050408,-0.022213,0.349076,-0.026216,-0.07948,0.144007,-0.043277,-0.06661,-0.162873,...,0.127269,-0.019724,0.18076,-0.46033,-0.145446,0.06009,0.125108,0.163428,0.012182,0.489622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,M10,0.058694,-0.018741,0.3504,-0.030028,-0.08478,0.145584,-0.041815,-0.067995,-0.179021,...,0.139139,-0.021686,0.204684,-0.439535,-0.061793,0.076704,0.116724,0.179552,0.017295,0.499338
57,M12,0.048354,-0.018524,0.348775,-0.026519,-0.078364,0.140046,-0.044458,-0.065258,-0.159179,...,0.12579,-0.018872,0.181405,-0.464125,-0.15211,0.077764,0.121129,0.160136,0.022099,0.480258
58,N1,0.060079,-0.025441,0.369751,-0.026726,-0.082967,0.136806,-0.044776,-0.068386,-0.180726,...,0.138927,-0.02376,0.201885,-0.429844,-0.116062,0.072199,0.111733,0.172374,0.002374,0.48556
59,N2,0.058958,-0.022816,0.344093,-0.025841,-0.080223,0.148451,-0.043178,-0.068162,-0.176436,...,0.129753,-0.021647,0.19837,-0.448173,-0.068692,0.067037,0.116078,0.183135,0.007686,0.503246


In [53]:
raw_data = pd.merge(all_data, df_evo.loc[:, :], how='left', on='id')

In [54]:
#extract X
col = raw_data.columns.values.tolist()
#col.remove('tag')
col.remove('UAA')
col.remove('label')
col.remove('id')
col.remove('ref:')
col.remove('dslf_fa13:')
col.remove('omega:')
col.remove('p_aa_pp:')
col.remove('cart_bonded:')
col.remove('rama_prepro:')

In [55]:
import numpy as np
X = np.array(raw_data[col])
y = np.array(raw_data['label'])

In [56]:
print(X.shape, np.sum(y))

(488, 1918) 19


In [57]:
#test_set = pd.DataFrame(X_test)
#test_set.insert(test_set.shape[1], "label", y_test)

In [58]:
import pickle

In [59]:
with open("FWD_evo"+evo_tag+"_test.pickle", 'wb') as f:
    pickle.dump([X, y], f)

In [60]:
#raw_data.to_excel("tmp.xlsx")